
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to configure the URL endpoint on cloudwatch alarm using unSkript legos.</b>
</div>

<br>

<center><h2>Configure url endpoint on a cloudwatch alarm</h2></center>

# Steps Overview
    Attach a webhook endpoint to one of the SNS attached to the AWS Cloudwatch alarm.

Here we will use unSkript Configure url endpoint on a cloudwatch alarm Lego. This lego takes alarm_name: str, region: str, url: str as input. This inputs is used to Configures the url endpoint to the SNS associated with a cloudwatch alarm.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import enum
from pydantic import BaseModel, Field
from typing import Optional, List
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from unskript.legos.aws.aws_get_handle import Session
from urllib.parse import urlparse

from beartype import beartype
@beartype
def aws_cloudwatch_attach_webhook_notification_to_alarm(
    hdl: Session,
    alarm_name: str,
    region: str,
    url: str
):
    cloudwatchClient = hdl.client("cloudwatch", region_name=region)

    # Get the configured SNS(es) to this alarm.
    alarmDetail = cloudwatchClient.describe_alarms(
        AlarmNames=[alarm_name]
    )
    if alarmDetail is None:
        print(f'Alarm {alarm_name} not found in AWS region {region}')
        return
    # Need to get the AlarmActions from either composite or metric field.
    if len(alarmDetail['CompositeAlarms']) > 0:
        snses = alarmDetail['CompositeAlarms'][0]['AlarmActions']
    else:
        snses = alarmDetail['MetricAlarms'][0]['AlarmActions']

    #Pick any sns to configure the url endpoint.
    if len(snses) == 0:
        print(f'No SNS configured for alarm {alarm_name}')
        return

    snsArn = snses[0]
    print(f'Configuring url endpoint on SNS {snsArn}')

    snsClient = hdl.client('sns', region_name=region)
    # Figure out the protocol from the url
    try:
        parsedURL = urlparse(url)
    except Exception as e:
        print(f'Invalid URL {url}, {e}')
        return

    if parsedURL.scheme != 'http' and parsedURL.scheme != 'https':
        print(f'Invalid URL {url}')
        return

    protocol = parsedURL.scheme
    try:
       response = snsClient.subscribe(
            TopicArn=snsArn,
            Protocol=protocol,
            Endpoint=url,
            ReturnSubscriptionArn=True)
    except Exception as e:
        print(f'Subscribe to SNS topic arn {snsArn} failed, {e}')
        return
    subscriptionArn = response['SubscriptionArn']
    print(f'URL {url} subscribed to SNS {snsArn}, subscription ARN {subscriptionArn}')
    return subscriptionArn



task = Task(Workflow())
task.configure(inputParamsJson='''{
    "alarm_name": "AlarmName",
    "region": "Region",
    "url": "URL"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(aws_cloudwatch_attach_webhook_notification_to_alarm, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

### Conclusion
In this Runbook, we demonstrated the use of unSkript's AWS legos to Configures the url endpoint to the SNS associated with a cloudwatch alarm. To view the full platform capabilities of unSkript please visit https://us.app.unskript.io